# Reference 
For better understanding of the code, please refer to the following code: 
- `./Lab/Code/practice/RFM-Koggle.ipynb`

In [1]:
import os 
import findspark 
findspark.init()

# for sql
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col
from pyspark.sql.functions import col, max as spark_max, count, sum as spark_sum, datediff, lit, min as spark_min
from pyspark.sql.types import IntegerType

# for time 
import time 


In [2]:
# 可以改成 *.csv 
root = '../../../Data/eCommerce-behavior-data/*.csv'
spark = SparkSession.builder.appName('eCommerce').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/29 01:01:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

ecommerce = spark.read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv(root)

In [4]:
ecommerce.createOrReplaceTempView('ecommerce_2019_oct')

In [5]:
ecommerce.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [6]:
ecommerce.show(5)

+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 01:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 01:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 01:00:01|      view|  17302664|2053013553853497655|                NULL| creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 01:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 01:00:01|      view|   1004775|2053013555631882655|electronics.smart...|xiaomi

## Only Select Purchase Columns

In [7]:
ecommerce = ecommerce.filter(col("event_type") == "purchase")
ecommerce.show(5)

+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|2019-11-01 01:00:41|  purchase|  13200605|2053013557192163841|furniture.bedroom...|   NULL| 566.3|559368633|d6034fa2-41fb-4ac...|
|2019-11-01 01:01:04|  purchase|   1005161|2053013555631882655|electronics.smart...| xiaomi|211.92|513351129|e6b7ce9b-1938-4e2...|
|2019-11-01 01:04:51|  purchase|   1004856|2053013555631882655|electronics.smart...|samsung|128.42|562958505|0f039697-fedc-40f...|
|2019-11-01 01:05:34|  purchase|  26401669|2053013563651392361|                NULL|lucente|109.66|541854711|c41c44d5-ef9b-41b...|
|2019-11-01 01:06:33|  purchase|   1801881|2053013554415534427|electronics.video.tv

## Aggregate user_session
- the user may has made multiple purchases in the same session.

In [8]:
result = ecommerce.groupBy('user_session').agg(
    spark_max('event_time').alias('Date_order'),  # alias is rename function
    spark_max('user_id').alias('user_id'), 
    count('user_session').alias('order_count'), 
    spark_sum('price').alias('price')  
)
result.show(5)

+--------------------+-------------------+---------+-----------+------+
|        user_session|         Date_order|  user_id|order_count| price|
+--------------------+-------------------+---------+-----------+------+
|8693715b-2f32-462...|2019-11-01 02:42:46|558439221|          1|  66.9|
|ad3086a0-22ad-4da...|2019-11-01 02:53:29|521374214|          1|185.85|
|5b0a2af3-293f-4bd...|2019-11-01 03:16:26|547622783|          1|206.96|
|3e5e22a7-ffd8-402...|2019-11-01 04:13:37|516883277|          1|230.93|
|279f59b0-d73f-445...|2019-11-01 04:38:18|561621729|          1|458.28|
+--------------------+-------------------+---------+-----------+------+
only showing top 5 rows



# **RFM Analysis**
RFM is a method used for analyzing customer value. It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries.

RFM stands for the three dimensions:

* Recency – How recently did the customer purchase?
* Frequency – How often do they purchase?
* Monetary Value – How much do they spend?

source: [wikipedia](https://en.wikipedia.org/wiki/RFM_(market_research))

so we will make that 3 attribute Recency, Frequency, and Monetary

In [9]:
max_date = result.agg(spark_max('Date_order')).collect()[0][0]
print("The latest date:", max_date)

The latest date: 2019-12-01 00:59:44


The last date we have is 2019-12-01 so we will use date 2019-12-2 as reference

In [10]:
study_date = '2019-12-02'

# prepare data for calculating the recency 
result = result.withColumn('last_purchase', datediff(lit(study_date), col('Date_order')))

# show the result
result.show(5)

+--------------------+-------------------+---------+-----------+------+-------------+
|        user_session|         Date_order|  user_id|order_count| price|last_purchase|
+--------------------+-------------------+---------+-----------+------+-------------+
|8693715b-2f32-462...|2019-11-01 02:42:46|558439221|          1|  66.9|           31|
|ad3086a0-22ad-4da...|2019-11-01 02:53:29|521374214|          1|185.85|           31|
|5b0a2af3-293f-4bd...|2019-11-01 03:16:26|547622783|          1|206.96|           31|
|3e5e22a7-ffd8-402...|2019-11-01 04:13:37|516883277|          1|230.93|           31|
|279f59b0-d73f-445...|2019-11-01 04:38:18|561621729|          1|458.28|           31|
+--------------------+-------------------+---------+-----------+------+-------------+
only showing top 5 rows



In [12]:
# Calculate Recency、Frequency 和 Monetary
RFM_result = result.groupBy('user_id').agg(
    spark_min('last_purchase').alias('Recency'),
    count('user_id').alias('Frequency'),
    spark_sum('price').alias('Monetary')
)

# Show the result
RFM_result.show(10)

+---------+-------+---------+------------------+
|  user_id|Recency|Frequency|          Monetary|
+---------+-------+---------+------------------+
|555200774|     31|        1|             84.38|
|565415116|     15|        5|1717.3100000000002|
|513689946|     31|        1|             85.87|
|512755684|     31|        3|            348.76|
|514118467|     31|        3|            336.95|
|562104717|     31|        5|            1127.5|
|512867322|     26|       14|12368.779999999999|
|546129282|      4|        5|           4934.68|
|530628095|     31|        3|            446.22|
|543043463|     31|        2|            367.76|
+---------+-------+---------+------------------+
only showing top 10 rows



### About the warning. 
As indicated [here](https://stackoverflow.com/questions/41661849/spill-to-disk-and-shuffle-write-spark) this warning means that your RAM is full and that part of the RAM contents are moved to disk.

## Frequency 

In [13]:
RFM_result.describe('Frequency').show()

+-------+------------------+
|summary|         Frequency|
+-------+------------------+
|  count|            697470|
|   mean|2.0112090842616888|
| stddev| 2.979720130244585|
|    min|                 1|
|    max|               328|
+-------+------------------+



## Recency

In [14]:
RFM_result.describe('Recency').show()

+-------+------------------+
|summary|           Recency|
+-------+------------------+
|  count|            697470|
|   mean|26.546968328386885|
| stddev| 17.20991564155434|
|    min|                 1|
|    max|                62|
+-------+------------------+



## Monetary

In [15]:
RFM_result.describe('Monetary').show()

+-------+-----------------+
|summary|         Monetary|
+-------+-----------------+
|  count|           697470|
|   mean|724.2639723142205|
| stddev|2271.239864151195|
|    min|             0.77|
|    max|        344153.29|
+-------+-----------------+

